In [4]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re
from urllib.parse import urlparse

In [6]:
# URL da página principal
url = "https://www.sicredi.com.br/home/"

# Domínios permitidos
DOMINIOS_PERMITIDOS = ["sicredi.com.br", "fundacaosicredi.org.br"]

# Extensões de arquivos a ignorar
EXTENSOES_IGNORAR = [
    ".pdf", ".mp3", ".mp4", ".avi", ".mov", ".wmv",
    ".jpg", ".jpeg", ".png", ".gif", ".svg", ".ico",
    ".zip", ".rar", ".tar", ".gz", ".doc", ".docx",
    ".xls", ".xlsx", ".ppt", ".pptx", ".txt", ".csv"
]

# Verifica se a URL pertence a um domínio permitido
def dominio_permitido(url):
    try:
        dominio = urlparse(url).netloc
        return any(dominio.endswith(d) for d in DOMINIOS_PERMITIDOS)
    except:
        return False

# Verifica se o link é válido (não é arquivo e está em PT-BR)
def link_valido(link):
    try:
        # Extrai apenas o caminho da URL (sem query string ?xxx e sem fragmentos #xxx)
        path = urlparse(link).path.lower()

        # Ignorar arquivos se o caminho termina com uma extensão proibida
        if any(path.endswith(ext) for ext in EXTENSOES_IGNORAR):
            return False
        
        # Ignorar páginas em inglês ou espanhol (ajustável conforme padrão do site)
        if re.search(r"/en\b|/es\b|/es-|/en-", link, re.IGNORECASE):
            return False

        return True
    except:
        return False

# Função para extrair o conteúdo de uma página sem os textos dentro das tags <a> e <button>
def extrair_conteudo_sem_links_e_botoes(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Remove todas as tags <a> e <button>
            for tag in soup.find_all(['a', 'button']):
                tag.decompose()
            
            # Extrai o texto restante
            texto = soup.get_text(separator="\n", strip=True)
            
            # Remove slogans específicos
            texto = re.sub(r"\bcom quem contar\b", "", texto, flags=re.IGNORECASE)
            texto = re.sub(r"\bseja associado\b", "", texto, flags=re.IGNORECASE)
            
            # Remove linhas vazias
            linhas = [linha.strip() for linha in texto.splitlines() if linha.strip()]
            return "\n".join(linhas)
        else:
            print(f"Erro ao acessar {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return None

# Função para coletar todos os links de uma página
def coletar_links(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            
            # Converte para links absolutos e filtra
            links_completos = [link for link in links if link.startswith('http')]
            
            # Filtra: domínio permitido + link válido
            links_filtrados = [
                link for link in links_completos
                if dominio_permitido(link) and link_valido(link)
            ]
            
            return list(set(links_filtrados))  # Remove duplicados
        else:
            print(f"Erro ao acessar {url}. Status code: {response.status_code}")
            return []
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return []

# Função principal para coletar dados de todas as páginas
def coletar_dados_pagina_principal(url):
    links = coletar_links(url)
    print(f"Encontrados {len(links)} links únicos na página principal.")

    visitados = set()        # Links já acessados
    conteudos_unicos = {}    # Conteúdo por URL

    while links:
        link = links.pop()
        
        # Pula se já foi visitado ou não pertence aos domínios permitidos
        if link in visitados or not dominio_permitido(link) or not link_valido(link):
            continue

        print(f"Acessando: {link}")
        visitados.add(link)  # Marca como visitado

        conteudo = extrair_conteudo_sem_links_e_botoes(link)
        new_links = coletar_links(link)

        # Adiciona só links válidos
        for l in new_links:
            if dominio_permitido(l) and link_valido(l) and l not in visitados:
                links.append(l)

        # Armazena conteúdo se for único
        if conteudo and conteudo not in conteudos_unicos.values():
            conteudos_unicos[link] = conteudo

        time.sleep(2)  # Boa prática para não sobrecarregar o servidor

    # Salvar os dados em JSON
    dados = [{'url': url, 'conteudo': conteudo} for url, conteudo in conteudos_unicos.items()]
    with open('conteudo_paginas_sem_links.json', 'w', encoding='utf-8') as file:
        json.dump(dados, file, ensure_ascii=False, indent=4)

    print("Dados tratados e salvos com sucesso!")


In [7]:
# Chama a função principal
coletar_dados_pagina_principal(url)

Encontrados 82 links únicos na página principal.
Acessando: https://www.sicredi.com.br/site/pix/
Acessando: https://www.sicredi.com.br/site/seguranca/
Acessando: https://www.sicredi.com.br/site/fundacao/
Acessando: https://www.sicredi.com.br/site/fundacao/comite-inclusao-diversidade-equidade/
Acessando: https://www.sicredi.com.br/nacomunidade/
Acessando: https://www.sicredi.com.br/trajetoria/
Acessando: https://www.sicredi.com.br/site/credito/para-agronegocio/
Acessando: https://www.sicredi.com.br/site/previdencia
Acessando: https://www.sicredi.com.br/site/credito/para-empresa/
Acessando: https://www.sicredi.com.br/site/atendimento/canais/fale-conosco/
Acessando: https://www.sicredi.com.br/site/shopping-do-sicredi/
Acessando: https://www.sicredi.com.br/segundaviaboleto/
Acessando: https://www.sicredi.com.br/site/sobre-nos/
Acessando: https://www.sicredi.com.br/site/blog/
Acessando: https://www.sicredi.com.br/site/leiloes-creditos-emergenciais/
Acessando: https://www.sicredi.com.br/site

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Acessando: https://www.sicredi.com.br/coop/norte/
Acessando: https://www.sicredi.com.br/site/investimentos/poupanca
Acessando: https://www.sicredi.com.br/coop/norte-sul/
Erro ao acessar https://www.sicredi.com.br/coop/norte-sul/. Status code: 404
Erro ao acessar https://www.sicredi.com.br/coop/norte-sul/. Status code: 404
Acessando: https://www.sicredi.com.br/coop/regiaocentro/
Acessando: https://www.sicredi.com.br/coop/regiaocentro/trabalhe-conosco-sicrediregiaocentro/
Acessando: https://www.sicredi.com.br/coop/regiaocentro/documentos-e-relatorios
Acessando: https://www.sicredi.com.br/coop/regiaocentro/governanca-cooperativa/
Acessando: https://www.sicredi.com.br/nacomunidade/patrocinio
Erro ao acessar https://www.sicredi.com.br/nacomunidade/patrocinio. Status code: 404
Erro ao acessar https://www.sicredi.com.br/nacomunidade/patrocinio. Status code: 404
Acessando: https://www.sicredi.com.br/coop/regiaocentro/estacao-sicredi
Acessando: https://www.sicredi.com.br/coop/regiaocentro/nosso